# Import Dependencies

In [ ]:
import pandas as pd
import nltk
from tqdm import tqdm
import re
import ast

from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.util import ngrams

import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\farha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Load data

In [32]:
df = pd.read_csv('data/PPN_clean.csv')
df.head()

,full_text,lower_case,remove_slang,remove_elongation,emoji_to_text,tweet_tokens,stem,remove_stopwords,remove_stop_words_nltk
0,@likromiahooy Paginya ppn naik 12%,@likromiahooy paginya ppn naik 12,@likromiahooy paginya ppn naik 12,@likromiahooy paginya ppn naik 12,@likromiahooy paginya ppn naik 12,"['paginya', 'ppn', 'naik', '12']","['pagi', 'ppn', 'naik', '12']","['pagi', 'ppn', '12']","['pagi', 'ppn', '12']"
1,@AdamVelcro 1. Krn sudah dikondisikan di media...,@adamvelcro 1 krn sudah dikondisikan di media ...,@adamvelcro 1 karena sudah dikondisikan di med...,@adamvelcro 1 karena sudah dikondisikan di med...,@adamvelcro 1 karena sudah dikondisikan di med...,"['1', 'karena', 'sudah', 'dikondisikan', 'di',...","['1', 'karena', 'sudah', 'kondisi', 'di', 'med...","['1', 'kondisi', 'media', 'kenaikan', 'ppn', '...","['1', 'kondisi', 'media', 'kenaikan', 'ppn', '..."
2,@exhumaclown rapat ppn krn naik jadi 12% kah?,@exhumaclown rapat ppn krn naik jadi 12 kah,@exhumaclown rapat ppn karena naik jadi 12 kah,@exhumaclown rapat ppn karena naik jadi 12 kah,@exhumaclown rapat ppn karena naik jadi 12 kah,"['rapat', 'ppn', 'karena', 'naik', 'jadi', '12...","['rapat', 'ppn', 'karena', 'naik', 'jadi', '12...","['rapat', 'ppn', '12', 'kah']","['rapat', 'ppn', '12']"
3,Apakah ini hari pemborosan sebelum ppn naik 12%?,apakah ini hari pemborosan sebelum ppn naik 12,apakah ini hari pemborosan sebelum ppn naik 12,apakah ini hari pemborosan sebelum ppn naik 12,apakah ini hari pemborosan sebelum ppn naik 12,"['apakah', 'ini', 'hari', 'pemborosan', 'sebel...","['apakah', 'ini', 'hari', 'boros', 'belum', 'p...","['boros', 'ppn', '12']","['boros', 'ppn', '12']"
4,@kompascom Kl pabrik / usaha nya yg memproduks...,@kompascom kl pabrik usaha nya yg memproduksi...,@kompascom kalo pabrik usaha nya yang memprod...,@kompascom kalo pabrik usaha nya yang memprod...,@kompascom kalo pabrik usaha nya yang memprod...,"['kalo', 'pabrik', 'usaha', 'nya', 'yang', 'me...","['kalau', 'pabrik', 'usaha', 'nya', 'yang', 'm...","['pabrik', 'usaha', 'nya', 'memproduksi', 'pro...","['pabrik', 'usaha', 'memproduksi', 'produk', '..."


get subset

In [42]:
corpus  = df[['remove_stop_words_nltk']].copy()
corpus

,remove_stop_words_nltk
0,"['pagi', 'ppn', '12']"
1,"['1', 'kondisi', 'media', 'kenaikan', 'ppn', '..."
2,"['rapat', 'ppn', '12']"
3,"['boros', 'ppn', '12']"
4,"['pabrik', 'usaha', 'memproduksi', 'produk', '..."
...,...
306,"['pasrah', 'nrimo', 'ing', 'pandum', 'nek', 'p..."
307,"['bilang', 'ppn', '12', 'sasar', 'barang', 'me..."
308,"['ppn', '12', 'tanda', 'kenaikan', 'duit', '12']"
309,"['ppn', '12', 'duit', 'negara', 'masyarakat', ..."


In [ ]:
corpus['remove_stop_words_nltk'] = corpus['remove_stop_words_nltk'].apply(lambda x: ast.literal_eval(x))
corpus

,remove_stop_words_nltk
0,"[pagi, ppn, 12]"
1,"[1, kondisi, media, kenaikan, ppn, 12, inisias..."
2,"[rapat, ppn, 12]"
3,"[boros, ppn, 12]"
4,"[pabrik, usaha, memproduksi, produk, pangan, k..."
...,...
306,"[pasrah, nrimo, ing, pandum, nek, ppn, 12, hore]"
307,"[bilang, ppn, 12, sasar, barang, mewah, doang,..."
308,"[ppn, 12, tanda, kenaikan, duit, 12]"
309,"[ppn, 12, duit, negara, masyarakat, potong, pr..."


In [45]:
corpus['string'] = corpus['remove_stop_words_nltk'].apply(lambda x: ' '.join(x))
corpus

,remove_stop_words_nltk,string
0,"[pagi, ppn, 12]",pagi ppn 12
1,"[1, kondisi, media, kenaikan, ppn, 12, inisias...",1 kondisi media kenaikan ppn 12 inisiasi pdip ...
2,"[rapat, ppn, 12]",rapat ppn 12
3,"[boros, ppn, 12]",boros ppn 12
4,"[pabrik, usaha, memproduksi, produk, pangan, k...",pabrik usaha memproduksi produk pangan kena pp...
...,...,...
306,"[pasrah, nrimo, ing, pandum, nek, ppn, 12, hore]",pasrah nrimo ing pandum nek ppn 12 hore
307,"[bilang, ppn, 12, sasar, barang, mewah, doang,...",bilang ppn 12 sasar barang mewah doang baca ha...
308,"[ppn, 12, tanda, kenaikan, duit, 12]",ppn 12 tanda kenaikan duit 12
309,"[ppn, 12, duit, negara, masyarakat, potong, pr...",ppn 12 duit negara masyarakat potong program a...


#

# Bag of Words (BoW)

A Bag of words is a an approach to transform text into numerical representation

- Describes the occurrence of words within a document or a collection of documents (corpus)
- Builds a vocabulary of the words and a measure of their presence


In [ ]:
vectorizer = CountVectorizer()
X_bow = vectorizer.fit_transform(corpus[''])
pd.DataFrame(X_bow.toarray(), columns=vectorizer.get_feature_names_out())

,011,012,02,021,022,03,08133099932,10,100,1000,...,xmgh,yaelah,yalon,yaudahlah,yes,you,yourself,zaman,zonk,zulhas
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
307,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
308,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
309,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# TF-IDF (Term Frequency - Inverse Document Frequency)


In [ ]:
vectorizer_tfidf = TfidfVectorizer()
X_tfidf = vectorizer_tfidf.fit_transform()
pd.DataFrame(X_tfidf.toarray(), columns=vectorizer_tfidf.get_feature_names_out())

ValueError: empty vocabulary; perhaps the documents only contain stop words